# HAR CNN training 

In [1]:
# Imports
import numpy as np
import os
from utils.semg_utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare data

In [2]:
path = os.path.abspath(os.getcwd())
X_train, labels_train, list_ch_train = read_data(data_path= path + os.sep + "sEMG" + os.sep + "Database 2" + os.sep, split_type="train") # train
X_test, labels_test, list_ch_test = read_data(data_path= path + os.sep + "sEMG" + os.sep + "Database 2" + os.sep, split_type="test") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

[1 1 1 ... 6 6 6]
(1800,)
(1800, 2496, 2)
(1800,)
[4. 6. 2. ... 2. 4. 5.]
[1 1 1 ... 6 6 6]
(1800,)
(1800, 2496, 2)
(1800,)
[5. 1. 1. ... 2. 2. 4.]


In [3]:
# Normalize?
X_train, X_test = standardize(X_train, X_test)

Train/Validation Split

In [4]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train, random_state = 123)

One-hot encoding:

In [5]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

In [6]:
# Imports
import tensorflow as tf

### Hyperparameters

In [7]:
batch_size = 100       # Batch size
seq_len = 2496          # Number of steps
learning_rate = 0.00001
epochs = 1000

n_classes = 6
n_channels = 2

### Construct the graph
Placeholders

In [8]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Build Convolutional Layers

Note: Should we use a different activation? Like tf.nn.tanh?

In [9]:
with graph.as_default():
    # (batch, 2496, 2) --> (batch, 1248, 4)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=4, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 1248, 4) --> (batch, 624, 8)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=8, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 624, 8) --> (batch, 312, 16)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=16, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 312, 16) --> (batch, 156, 32)
    conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=32, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')
    
    # (batch, 156, 32) --> (batch, 78, 64)
    conv5 = tf.layers.conv1d(inputs=max_pool_4, filters=64, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_5 = tf.layers.max_pooling1d(inputs=conv5, pool_size=2, strides=2, padding='same')
    
    # (batch, 78, 64) --> (batch, 39, 128)
    conv6 = tf.layers.conv1d(inputs=max_pool_5, filters=128, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_6 = tf.layers.max_pooling1d(inputs=conv6, pool_size=2, strides=2, padding='same')
    

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


Now, flatten and pass to the classifier

In [10]:
with graph.as_default():
    # Flatten and add dropout
    flat = tf.reshape(max_pool_6, (-1, 128 * 39))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



### Train the network

In [11]:
if (os.path.exists('checkpoints-cnn') == False):
    !mkdir checkpoints-cnn

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
                    # Loss
                    loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/har.ckpt")

Epoch: 0/1000 Iteration: 5 Train loss: 1.751210 Train acc: 0.220000
Epoch: 0/1000 Iteration: 10 Train loss: 1.795625 Train acc: 0.150000
Epoch: 0/1000 Iteration: 10 Validation loss: 1.765296 Validation acc: 0.160000
Epoch: 1/1000 Iteration: 15 Train loss: 1.788684 Train acc: 0.170000
Epoch: 1/1000 Iteration: 20 Train loss: 1.775067 Train acc: 0.150000
Epoch: 1/1000 Iteration: 20 Validation loss: 1.759932 Validation acc: 0.157500
Epoch: 1/1000 Iteration: 25 Train loss: 1.755172 Train acc: 0.200000
Epoch: 2/1000 Iteration: 30 Train loss: 1.789539 Train acc: 0.120000
Epoch: 2/1000 Iteration: 30 Validation loss: 1.754913 Validation acc: 0.157500
Epoch: 2/1000 Iteration: 35 Train loss: 1.758295 Train acc: 0.170000
Epoch: 3/1000 Iteration: 40 Train loss: 1.765831 Train acc: 0.190000
Epoch: 3/1000 Iteration: 40 Validation loss: 1.750312 Validation acc: 0.157500
Epoch: 3/1000 Iteration: 45 Train loss: 1.718679 Train acc: 0.140000
Epoch: 3/1000 Iteration: 50 Train loss: 1.745513 Train acc: 0.22

Epoch: 29/1000 Iteration: 380 Train loss: 1.651099 Train acc: 0.230000
Epoch: 29/1000 Iteration: 380 Validation loss: 1.667621 Validation acc: 0.235000
Epoch: 29/1000 Iteration: 385 Train loss: 1.617059 Train acc: 0.280000
Epoch: 29/1000 Iteration: 390 Train loss: 1.629548 Train acc: 0.270000
Epoch: 29/1000 Iteration: 390 Validation loss: 1.665566 Validation acc: 0.235000
Epoch: 30/1000 Iteration: 395 Train loss: 1.670549 Train acc: 0.210000
Epoch: 30/1000 Iteration: 400 Train loss: 1.665624 Train acc: 0.220000
Epoch: 30/1000 Iteration: 400 Validation loss: 1.663496 Validation acc: 0.235000
Epoch: 31/1000 Iteration: 405 Train loss: 1.714853 Train acc: 0.200000
Epoch: 31/1000 Iteration: 410 Train loss: 1.704064 Train acc: 0.220000
Epoch: 31/1000 Iteration: 410 Validation loss: 1.661416 Validation acc: 0.232500
Epoch: 31/1000 Iteration: 415 Train loss: 1.653892 Train acc: 0.290000
Epoch: 32/1000 Iteration: 420 Train loss: 1.717153 Train acc: 0.170000
Epoch: 32/1000 Iteration: 420 Validat

Epoch: 57/1000 Iteration: 750 Train loss: 1.607475 Train acc: 0.190000
Epoch: 57/1000 Iteration: 750 Validation loss: 1.578178 Validation acc: 0.277500
Epoch: 58/1000 Iteration: 755 Train loss: 1.593503 Train acc: 0.270000
Epoch: 58/1000 Iteration: 760 Train loss: 1.481300 Train acc: 0.390000
Epoch: 58/1000 Iteration: 760 Validation loss: 1.575127 Validation acc: 0.280000
Epoch: 58/1000 Iteration: 765 Train loss: 1.578242 Train acc: 0.320000
Epoch: 59/1000 Iteration: 770 Train loss: 1.569010 Train acc: 0.300000
Epoch: 59/1000 Iteration: 770 Validation loss: 1.572014 Validation acc: 0.280000
Epoch: 59/1000 Iteration: 775 Train loss: 1.505770 Train acc: 0.350000
Epoch: 59/1000 Iteration: 780 Train loss: 1.540151 Train acc: 0.310000
Epoch: 59/1000 Iteration: 780 Validation loss: 1.568952 Validation acc: 0.282500
Epoch: 60/1000 Iteration: 785 Train loss: 1.560796 Train acc: 0.260000
Epoch: 60/1000 Iteration: 790 Train loss: 1.538315 Train acc: 0.320000
Epoch: 60/1000 Iteration: 790 Validat

Epoch: 85/1000 Iteration: 1115 Train loss: 1.472868 Train acc: 0.370000
Epoch: 86/1000 Iteration: 1120 Train loss: 1.508541 Train acc: 0.350000
Epoch: 86/1000 Iteration: 1120 Validation loss: 1.445545 Validation acc: 0.430000
Epoch: 86/1000 Iteration: 1125 Train loss: 1.520892 Train acc: 0.380000
Epoch: 86/1000 Iteration: 1130 Train loss: 1.437819 Train acc: 0.440000
Epoch: 86/1000 Iteration: 1130 Validation loss: 1.441311 Validation acc: 0.427500
Epoch: 87/1000 Iteration: 1135 Train loss: 1.540110 Train acc: 0.360000
Epoch: 87/1000 Iteration: 1140 Train loss: 1.441469 Train acc: 0.410000
Epoch: 87/1000 Iteration: 1140 Validation loss: 1.437094 Validation acc: 0.435000
Epoch: 88/1000 Iteration: 1145 Train loss: 1.452198 Train acc: 0.370000
Epoch: 88/1000 Iteration: 1150 Train loss: 1.354704 Train acc: 0.440000
Epoch: 88/1000 Iteration: 1150 Validation loss: 1.432823 Validation acc: 0.447500
Epoch: 88/1000 Iteration: 1155 Train loss: 1.486917 Train acc: 0.480000
Epoch: 89/1000 Iteration

Epoch: 113/1000 Iteration: 1480 Train loss: 1.386177 Train acc: 0.550000
Epoch: 113/1000 Iteration: 1480 Validation loss: 1.275059 Validation acc: 0.597500
Epoch: 114/1000 Iteration: 1485 Train loss: 1.250190 Train acc: 0.630000
Epoch: 114/1000 Iteration: 1490 Train loss: 1.269697 Train acc: 0.530000
Epoch: 114/1000 Iteration: 1490 Validation loss: 1.270015 Validation acc: 0.600000
Epoch: 114/1000 Iteration: 1495 Train loss: 1.154159 Train acc: 0.610000
Epoch: 115/1000 Iteration: 1500 Train loss: 1.233755 Train acc: 0.630000
Epoch: 115/1000 Iteration: 1500 Validation loss: 1.264930 Validation acc: 0.597500
Epoch: 115/1000 Iteration: 1505 Train loss: 1.323674 Train acc: 0.470000
Epoch: 116/1000 Iteration: 1510 Train loss: 1.369507 Train acc: 0.500000
Epoch: 116/1000 Iteration: 1510 Validation loss: 1.259829 Validation acc: 0.600000
Epoch: 116/1000 Iteration: 1515 Train loss: 1.410363 Train acc: 0.430000
Epoch: 116/1000 Iteration: 1520 Train loss: 1.303128 Train acc: 0.500000
Epoch: 116/

...

Epoch: 363/1000 Iteration: 4720 Train loss: 0.556998 Train acc: 0.790000
Epoch: 363/1000 Iteration: 4720 Validation loss: 0.600884 Validation acc: 0.795000
Epoch: 363/1000 Iteration: 4725 Train loss: 0.549962 Train acc: 0.780000
Epoch: 363/1000 Iteration: 4730 Train loss: 0.676945 Train acc: 0.790000
Epoch: 363/1000 Iteration: 4730 Validation loss: 0.600548 Validation acc: 0.792500
Epoch: 364/1000 Iteration: 4735 Train loss: 0.467731 Train acc: 0.850000
Epoch: 364/1000 Iteration: 4740 Train loss: 0.533228 Train acc: 0.810000
Epoch: 364/1000 Iteration: 4740 Validation loss: 0.600094 Validation acc: 0.795000
Epoch: 364/1000 Iteration: 4745 Train loss: 0.455073 Train acc: 0.820000
Epoch: 365/1000 Iteration: 4750 Train loss: 0.609483 Train acc: 0.820000
Epoch: 365/1000 Iteration: 4750 Validation loss: 0.599520 Validation acc: 0.792500
Epoch: 365/1000 Iteration: 4755 Train loss: 0.551666 Train acc: 0.830000
Epoch: 366/1000 Iteration: 4760 Train loss: 0.556967 Train acc: 0.810000
Epoch: 366/

Epoch: 390/1000 Iteration: 5080 Train loss: 0.508481 Train acc: 0.820000
Epoch: 390/1000 Iteration: 5080 Validation loss: 0.582098 Validation acc: 0.802500
Epoch: 391/1000 Iteration: 5085 Train loss: 0.524067 Train acc: 0.840000
Epoch: 391/1000 Iteration: 5090 Train loss: 0.643409 Train acc: 0.730000
Epoch: 391/1000 Iteration: 5090 Validation loss: 0.581037 Validation acc: 0.807500
Epoch: 391/1000 Iteration: 5095 Train loss: 0.495104 Train acc: 0.810000
Epoch: 392/1000 Iteration: 5100 Train loss: 0.632453 Train acc: 0.770000
Epoch: 392/1000 Iteration: 5100 Validation loss: 0.580330 Validation acc: 0.807500
Epoch: 392/1000 Iteration: 5105 Train loss: 0.558215 Train acc: 0.760000
Epoch: 393/1000 Iteration: 5110 Train loss: 0.462316 Train acc: 0.840000
Epoch: 393/1000 Iteration: 5110 Validation loss: 0.580150 Validation acc: 0.807500
Epoch: 393/1000 Iteration: 5115 Train loss: 0.518363 Train acc: 0.800000
Epoch: 393/1000 Iteration: 5120 Train loss: 0.676722 Train acc: 0.760000
Epoch: 393/

Epoch: 418/1000 Iteration: 5440 Train loss: 0.449303 Train acc: 0.840000
Epoch: 418/1000 Iteration: 5440 Validation loss: 0.568358 Validation acc: 0.807500
Epoch: 418/1000 Iteration: 5445 Train loss: 0.706566 Train acc: 0.790000
Epoch: 419/1000 Iteration: 5450 Train loss: 0.403147 Train acc: 0.880000
Epoch: 419/1000 Iteration: 5450 Validation loss: 0.568049 Validation acc: 0.807500
Epoch: 419/1000 Iteration: 5455 Train loss: 0.550465 Train acc: 0.810000
Epoch: 419/1000 Iteration: 5460 Train loss: 0.419475 Train acc: 0.850000
Epoch: 419/1000 Iteration: 5460 Validation loss: 0.568362 Validation acc: 0.807500
Epoch: 420/1000 Iteration: 5465 Train loss: 0.585341 Train acc: 0.800000
Epoch: 420/1000 Iteration: 5470 Train loss: 0.468561 Train acc: 0.870000
Epoch: 420/1000 Iteration: 5470 Validation loss: 0.569334 Validation acc: 0.807500
Epoch: 421/1000 Iteration: 5475 Train loss: 0.538856 Train acc: 0.790000
Epoch: 421/1000 Iteration: 5480 Train loss: 0.601089 Train acc: 0.800000
Epoch: 421/

Epoch: 446/1000 Iteration: 5800 Train loss: 0.474963 Train acc: 0.820000
Epoch: 446/1000 Iteration: 5800 Validation loss: 0.559190 Validation acc: 0.815000
Epoch: 446/1000 Iteration: 5805 Train loss: 0.647150 Train acc: 0.730000
Epoch: 446/1000 Iteration: 5810 Train loss: 0.460605 Train acc: 0.840000
Epoch: 446/1000 Iteration: 5810 Validation loss: 0.558406 Validation acc: 0.812500
Epoch: 447/1000 Iteration: 5815 Train loss: 0.624770 Train acc: 0.710000
Epoch: 447/1000 Iteration: 5820 Train loss: 0.478747 Train acc: 0.820000
Epoch: 447/1000 Iteration: 5820 Validation loss: 0.556797 Validation acc: 0.817500
Epoch: 448/1000 Iteration: 5825 Train loss: 0.491741 Train acc: 0.840000
Epoch: 448/1000 Iteration: 5830 Train loss: 0.415941 Train acc: 0.840000
Epoch: 448/1000 Iteration: 5830 Validation loss: 0.556587 Validation acc: 0.817500
Epoch: 448/1000 Iteration: 5835 Train loss: 0.674664 Train acc: 0.780000
Epoch: 449/1000 Iteration: 5840 Train loss: 0.393086 Train acc: 0.860000
Epoch: 449/

Epoch: 473/1000 Iteration: 6160 Train loss: 0.667384 Train acc: 0.760000
Epoch: 473/1000 Iteration: 6160 Validation loss: 0.550648 Validation acc: 0.817500
Epoch: 474/1000 Iteration: 6165 Train loss: 0.346905 Train acc: 0.910000
Epoch: 474/1000 Iteration: 6170 Train loss: 0.485911 Train acc: 0.830000
Epoch: 474/1000 Iteration: 6170 Validation loss: 0.549486 Validation acc: 0.815000
Epoch: 474/1000 Iteration: 6175 Train loss: 0.363652 Train acc: 0.850000
Epoch: 475/1000 Iteration: 6180 Train loss: 0.587277 Train acc: 0.810000
Epoch: 475/1000 Iteration: 6180 Validation loss: 0.549040 Validation acc: 0.815000
Epoch: 475/1000 Iteration: 6185 Train loss: 0.438122 Train acc: 0.840000
Epoch: 476/1000 Iteration: 6190 Train loss: 0.467766 Train acc: 0.840000
Epoch: 476/1000 Iteration: 6190 Validation loss: 0.548460 Validation acc: 0.812500
Epoch: 476/1000 Iteration: 6195 Train loss: 0.660179 Train acc: 0.720000
Epoch: 476/1000 Iteration: 6200 Train loss: 0.444234 Train acc: 0.830000
Epoch: 476/

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1}
        
        batch_acc = sess.run(accuracy, feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))